In [159]:
import os
import glob
import numpy as np
import scipy.io as sio
from scipy.stats.stats import spearmanr, normaltest
from sklearn import preprocessing
import warnings
warnings.filterwarnings('ignore')

beta_dir = './beta'

SUBJECTS = [1, 2, 3, 4]
MODELS = ['dvae', 'vae']
THRESHOLD = 4
LATENT_DIMENSION = 24
INEFFECTIVE_THRESHOLD = .5

In [160]:
yj = preprocessing.PowerTransformer(method='yeo-johnson')

for subject_num in SUBJECTS:
    print('Subject {}'.format(subject_num))
    for model_type in MODELS:
        localizer_map = sio.loadmat('localizer-maps/sub-{:02d}_sig.mat'.format(subject_num))['data'][0]
        above_threshold = localizer_map > THRESHOLD
        n_voxels = np.sum(above_threshold)
        #print('Number above threshold: {}'.format(n_voxels))
        betas = sio.loadmat('betas/sub{:02d}_{}-beta.mat'.format(subject_num, model_type))
        betas = np.array(betas['data'][above_threshold]).transpose()
        latent_betas = betas[:LATENT_DIMENSION]
        
        # Normalize the latent betas to N(0, 1)
        normalized_latent_betas = yj.fit_transform(latent_betas) #preprocessing.scale(latent_betas)
        
        # Remove the betas that are less than one std from the mean
        ineffective_betas = np.abs(normalized_latent_betas) < INEFFECTIVE_THRESHOLD
        normalized_latent_betas[ineffective_betas] = False
        
        # Sum the number of pairwise latent dimension overlaps
        total_overlap = 0
        for i in range(LATENT_DIMENSION):
            for j in range(i+1, LATENT_DIMENSION):
                overlap = np.sum(np.logical_and(normalized_latent_betas[i], normalized_latent_betas[j]))
                total_overlap += overlap
                #print('({},{}): {}'.format(i,j, overlap))
                
        print('{} overlap: {}'.format(model_type, total_overlap))
            

Subject 1
dvae overlap: 84253
vae overlap: 89619
Subject 2
dvae overlap: 211185
vae overlap: 233210
Subject 3
dvae overlap: 748002
vae overlap: 730607
Subject 4
dvae overlap: 204446
vae overlap: 220510


In [157]:
qt = preprocessing.QuantileTransformer(n_quantiles=50000, output_distribution='normal')
bc = preprocessing.PowerTransformer(method='box-cox')
yj = preprocessing.PowerTransformer(method='yeo-johnson')
a = yj.fit_transform(latent_betas)
print(np.mean(a))
print(np.std(a))
print(latent_betas[0][:20])
print(a[0][:20])

4.6144863e-10
1.0
[-0.0199232  -0.01522118 -0.0105066  -0.0094768  -0.00486771 -0.00477402
 -0.02081244 -0.01842963 -0.01352152 -0.01390422 -0.01647327 -0.0161181
 -0.00900957 -0.01936265 -0.01800413 -0.01868273 -0.00458421 -0.02107806
 -0.02013998 -0.01950914]
[-2.2367926  -2.195705   -1.652713   -1.8701108  -0.98084015 -1.0620736
 -2.270307   -2.320767   -1.3271359  -2.410403   -1.5052649  -1.4843676
 -2.0929558  -1.8635281  -1.6541994  -1.673396   -1.0034643  -2.094548
 -1.8782969  -1.7457006 ]
